In [377]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp

#read in data
testResults=pd.read_csv('test_results.csv')
userTable=pd.read_csv('user_table.csv')
#userTable is shorter than testResults, so can only do geographic tests on a smaller sample
#remove linux users: none converted in test condition
newResults=testResults.drop(testResults.index[testResults.operative_system=='linux'])
#merge into same table
geoData=pd.merge(newResults,userTable,how='right',on='user_id')

#tested source, device, operative_system: p-value >0.98 for all
testab=geoData[['test','converted','source','user_id']].groupby(['test','converted','source']).agg(['count'])
testab

user_id
                                 count
test converted source                 
0    0         ads-bing          12389
               ads-google        31752
               ads-yahoo          4045
               ads_facebook      28609
               ads_other         16384
               direct_traffic    33014
               friend_referral   10834
               seo-bing           1122
               seo-google        12778
               seo-other          4983
               seo-yahoo          3703
               seo_facebook      11425
     1         ads-bing            168
               ads-google          735
               ads-yahoo            66
               ads_facebook        694
               ads_other           249
               direct_traffic      451
               friend_referral     466
               seo-bing             32
               seo-google          229
               seo-other            91
               seo-yahoo            75
               seo_facebook        202
1    0         ads-bing           7049
               ads-google        17960
               ads-yahoo          2360
               ads_facebook      16225
               ads_other          8984
               direct_traffic    18312
               friend_referral    6160
               seo-bing            700
               seo-google         6801
               seo-other          2839
               seo-yahoo          2069
               seo_facebook       6497
     1         ads-bing             70
               ads-google          361
               ads-yahoo            30
               ads_facebook        285
               ads_other           121
               direct_traffic      184
               friend_referral     210
               seo-bing              8
               seo-google          111
               seo-other            36
               seo-yahoo            23
               seo_facebook         91

In [384]:
testCounts=testab['user_id'].groupby(level=[0,2])
totalCts=testCounts.sum()['count']

testabPcts=[]
for test in [0,1]:
    temp=[]
    #look at difference in converted rates
    for catVar in range(geoData['source'].nunique()):
        temp.append(float(testab['user_id']['count'][test][1][catVar])/float(totalCts[test][catVar]))
    testabPcts.append(temp)

testworked=[testabPcts[0][i]<testabPcts[1][i] for i in range(len(testabPcts[0]))]
testworked

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

## If any row above is True, then that subsection of the sample showed that the price reduction test worked. So far, none show this. Check p-value below.

In [386]:
from scipy import stats
sp.stats.chisquare(testabPcts[1], testabPcts[0])

#no significance to the result though. why p-value=nan when using df=number of samples?

Power_divergenceResult(statistic=0.022780088664789002, pvalue=0.99999999999992961)